In [378]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from datetime import datetime
from tkcalendar import DateEntry
from tkinter import constants as tk_constants
import sqlite3
db_path = os.path.join(os.getenv("APPDATA"), r"C:\Users\DELL8\OneDrive\Pictures\DB.Browser.for.SQLite-v3.13.1-win64\stkmanage.db")
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
def initialize_database():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Create 'transactions' table
    cursor.execute('''CREATE TABLE IF NOT EXISTS transactions (
                        item_id INTEGER PRIMARY KEY AUTOINCREMENT,
                        item_name VARCHAR NOT NULL,
                        party_name VARCHAR NOT NULL,
                        item_date DATE NOT NULL,
                        item_size VARCHAR NOT NULL,
                        item_in_stock INTEGER NOT NULL,
                        item_out_stock INTEGER NOT NULL,
                        balance INTEGER NOT NULL
                      )''')
    
    # Create 'stock' table
    cursor.execute('''CREATE TABLE IF NOT EXISTS stock (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        item_name TEXT NOT NULL,
                        quantity INTEGER NOT NULL,
                        date_added TEXT NOT NULL
                      )''')
    
    conn.commit()
    conn.close()
initialize_database()
today = datetime.today().strftime('%d-%m-%Y')

In [379]:
def update_out_stock_state(*args):
    if in_stock_var.get() == '':
        out_stock_entry.config(state=tk.NORMAL)
    else:
        in_stock_value = int(in_stock_var.get())
        out_stock_entry.config(state=tk.NORMAL if in_stock_value == 0 else tk.DISABLED)

    if out_stock_var.get() == '':
        in_stock_entry.config(state=tk.NORMAL)
    else:
        out_stock_value = int(out_stock_var.get())
        in_stock_entry.config(state=tk.NORMAL if out_stock_value == 0 else tk.DISABLED)

In [380]:
def load_transactions():
    conn = None  
    try:
        conn = sqlite3.connect(db_path)  
        cursor = conn.cursor()

        cursor.execute("""
            SELECT item_name, party_name, item_date, item_size, 
                   item_in_stock, item_out_stock, balance 
            FROM transactions ORDER BY item_id ASC
        """)
        transactions = cursor.fetchall()
        tree.delete(*tree.get_children()) 

        for transaction in transactions:
            tree.insert("", "end", values=transaction)

    except sqlite3.Error as e:
        messagebox.showerror("Database Error", f"Error: {e}")

    finally:
        if conn is not None: 
            conn.close()


In [381]:
def update_out_stock_transact():
    try:
        conn = sqlite3.connect(db_path) 
        cursor = conn.cursor()
        
        cursor.execute("SELECT item_name, party_name, item_date, item_size, item_in_stock, item_out_stock, balance FROM transactions WHERE item_out_stock > 0 AND item_date = ?", (today,))
        transactions = cursor.fetchall()

        for item in tree.get_children():
            tree.delete(item)
        
        if not transactions:
            messagebox.showinfo("Info", "No out-stock transactions found for today.")
            return  # Exit function early if no data

        for transaction in transactions:
            tree.insert("", "end", values=(
                transaction[0],  # item_name
                transaction[1],  # party_name
                transaction[2],  # item_date
                transaction[3],  # item_size
                transaction[4],  # item_in_stock
                transaction[5],  # item_out_stock
                transaction[6]   # balance
            ))

    except sqlite3.Error as e:
        messagebox.showerror("Database Error", str(e))

    finally:
        conn.close()


In [382]:
def update_in_stock_transact():
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT item_name, party_name, item_date, item_size, item_in_stock, item_out_stock, balance FROM transactions WHERE item_in_stock > 0 AND item_date = ?", (today,))
        transactions = cursor.fetchall()

        for item in tree.get_children():
            tree.delete(item)

        if not transactions:
            messagebox.showinfo("Info", "No in-stock transactions found for today.")
            return  # Exit function early if no data

        for transaction in transactions:
            tree.insert("", "end", values=(
                transaction[0],  # item_name
                transaction[1],  # party_name
                transaction[2],  # item_date
                transaction[3],  # item_size
                transaction[4],  # item_in_stock
                transaction[5],  # item_out_stock
                transaction[6]   # balance
            ))

    except sqlite3.Error as e:
        messagebox.showerror("Database Error", str(e))

    finally:
        conn.close()


In [383]:
def on_tree_select(event):
    selected_item = tree.selection()
    if selected_item:
        item = tree.item(selected_item, "values")
        cal.set_date(item[2])
        item_name_entry.delete(0, tk.END)
        item_name_entry.insert(0, item[0])
        party_name_entry.delete(0, tk.END)
        party_name_entry.insert(0, item[1])
        size_combobox.set(item[3])
        in_stock_var.set(item[4])
        out_stock_var.set(item[5])

In [384]:
def fetch_dropdown_values():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute("SELECT DISTINCT item_name FROM transactions")
    items = [row[0] for row in cursor.fetchall()]
    
    cursor.execute("SELECT DISTINCT party_name FROM transactions")
    parties = [row[0] for row in cursor.fetchall()]
    
    conn.close()
    
    return items, parties

In [385]:
import sqlite3
from tkinter import messagebox

def submit_form():
    date = cal.get_date().strftime("%d-%m-%Y")
    item_name = item_name_entry_combobox.get().strip().upper()
    party_name = party_name_entry_combobox.get().strip().upper()
    size = size_combobox.get().strip().upper()
    
    in_stock = int(in_stock_var.get()) if in_stock_var.get() else 0
    out_stock = int(out_stock_var.get()) if out_stock_var.get() else 0

    db_path = r"C:\Users\DELL8\OneDrive\Pictures\DB.Browser.for.SQLite-v3.13.1-win64\stkmanage.db"
    
    # Validate input fields
    if not item_name or not party_name or not size or (in_stock == 0 and out_stock == 0):
        messagebox.showerror("ERROR", "ENTER APPROPRIATE DATA")
        return

    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Check previous balance of the item
        cursor.execute("""
            SELECT balance FROM transactions 
            WHERE item_name = ? AND item_size = ? AND party_name = ? 
            ORDER BY item_id DESC LIMIT 1
        """, (item_name, size, party_name))
        
        transactions = cursor.fetchall()
        prev_balance = transactions[0][0] if transactions else 0

        # Calculate new balance
        if in_stock > 0:
            new_balance = prev_balance + in_stock
        elif out_stock > 0:
            if out_stock <= prev_balance:
                new_balance = prev_balance - out_stock
            else:
                messagebox.showerror("Error", "Not Sufficient Balance")
                return
        else:
            new_balance = prev_balance  # No change if both in_stock & out_stock are 0

        # Insert new transaction
        cursor.execute("""
            INSERT INTO transactions (item_name, party_name, item_date, item_size, item_in_stock, item_out_stock, balance)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (item_name, party_name, date, size, in_stock, out_stock, new_balance))

        conn.commit()
        messagebox.showinfo("Success", "Data Added Successfully")

    except Exception as e:
        print("Error submitting form:", e)
        messagebox.showerror("Error", f"Error submitting form: {e}")

    finally:
        conn.close()
        update_dropdowns()
        tree.delete(*tree.get_children())
        load_transactions()


In [386]:
def filter_transaction():
    print("Filter Transaction : ")

    selected_date = filter_date_entry.get_date().strftime("%d-%m-%Y")
    item_name = filter_item_name_entry.get().upper()

    print(selected_date, item_name)

    if(item_name != ''):
        try:
            db_path = r"C:\Users\DELL8\OneDrive\Pictures\DB.Browser.for.SQLite-v3.13.1-win64\stkmanage.db"
            conn = sqlite3.connect(db_path)
            cursor.execute("SELECT item_name, party_name ,item_date, item_size, item_in_stock, item_out_stock, balance FROM transactions WHERE item_name = ? AND item_date = ?", (item_name,selected_date,))
            transactions = cursor.fetchall()

            if(transactions != []):
                for item in tree.get_children():
                    tree.delete(item)
                
                for transaction in transactions:
                    tree.insert("", "end", values=transaction)
            else:
                messagebox.showerror("Error", "No Transactions")

        except Exception as e:
            print(e)
    else:
        messagebox.showerror("Error", "Enter the Item Name")

In [387]:
def load_all_transactions(tree):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        cursor.execute("""
            SELECT item_name, party_name, item_date, item_size, item_in_stock, item_out_stock, balance 
            FROM transactions 
            ORDER BY item_id ASC
        """)
        transactions = cursor.fetchall()

        for item in tree.get_children():
            tree.delete(item)

        if not transactions:
            messagebox.showinfo("Info", "No transactions found.")
            return  # Exit function early if no data

        for transaction in transactions:
            tree.insert("", "end", values=(
                transaction[0],  # item_name
                transaction[1],  # party_name
                transaction[2],  # item_date
                transaction[3],  # item_size
                transaction[4],  # item_in_stock
                transaction[5],  # item_out_stock
                transaction[6]   # balance
            ))

    except sqlite3.Error as e:
        messagebox.showerror("Database Error", str(e))

    finally:
        conn.close()


In [388]:
def checkSize(size):
    sizes = ['M', 'L', 'XL', 'XXL', '3XL', '4XL', '5XL', 'Free', 'Plus']
    index_size = sizes.index(size) + 3
    return index_size

In [389]:
def update_dropdowns():
    items, parties = fetch_dropdown_values()
    item_name_entry_combobox['values'] = items
    party_name_entry_combobox['values'] = parties

In [390]:
def purchase_report(tree, input_date):
    try:
        cursor.execute("SELECT item_name, party_name, item_date, item_size, item_in_stock, item_out_stock, balance FROM transactions WHERE item_date = ?", (input_date,))
        transactions = cursor.fetchall()

        if (transactions != []):
            data_format = [0,0,0,0,0,0,0,0,0,0,0,0,0]
            main_data = []
            unique_items =  []
            party_names = []
            for x in range(len(transactions)):
                if transactions[x][0] not in unique_items:
                    unique_items.append(transactions[x][0])
                if transactions[x][1] not in party_names:
                    party_names.append(transactions[x][1])

            for i in range(len(unique_items)):
                for j in range(len(party_names)):
                    for k in range(len(transactions)):
                        if((unique_items[i] == transactions[k][0]) and (party_names[j] == transactions[k][1]) and (transactions[k][4] > 0)):
                            size_index = checkSize(transactions[k][3])
                            data_format[size_index] = data_format[size_index] + transactions[k][4]
                            data_format[12] = data_format[12] + transactions[k][4]

                            data_format[0] = transactions[k][2]
                            data_format[1] = transactions[k][0]
                            data_format[2] = transactions[k][1]
                    if(data_format!=[0,0,0,0,0,0,0,0,0,0,0,0,0]):
                        main_data.append(data_format)
                    data_format = [0,0,0,0,0,0,0,0,0,0,0,0,0]

            for item in tree.get_children():
                tree.delete(item)

            for transaction in main_data:
                tree.insert("", "end", values=transaction)

        else:
            messagebox.showerror("Error", "No Transactions")
    
    except Exception as e:
        print("Error loading transactions:", e)

In [391]:
def sales_report(tree, input_date):
    try:
        cursor.execute("SELECT item_name, party_name, item_date, item_size, item_in_stock, item_out_stock, balance FROM transactions WHERE item_date = ?", (input_date,))
        transactions = cursor.fetchall()

        if (transactions != []):
            data_format = [0,0,0,0,0,0,0,0,0,0,0,0,0]
            main_data = []
            unique_items =  []
            party_names = []
            for x in range(len(transactions)):
                if transactions[x][0] not in unique_items:
                    unique_items.append(transactions[x][0])
                if transactions[x][1] not in party_names:
                    party_names.append(transactions[x][1])

            for i in range(len(unique_items)):
                for j in range(len(party_names)):
                    for k in range(len(transactions)):
                        if((unique_items[i] == transactions[k][0]) and (party_names[j] == transactions[k][1]) and (transactions[k][5] > 0)):
                            size_index = checkSize(transactions[k][3])
                            data_format[size_index] = data_format[size_index] + transactions[k][5]
                            data_format[12] = data_format[12] + transactions[k][5]

                            data_format[0] = transactions[k][2]
                            data_format[1] = transactions[k][0]
                            data_format[2] = transactions[k][1]
                    if(data_format!=[0,0,0,0,0,0,0,0,0,0,0,0,0]):
                        main_data.append(data_format)
                    data_format = [0,0,0,0,0,0,0,0,0,0,0,0,0]

            for item in tree.get_children():
                tree.delete(item)

            for transaction in main_data:
                tree.insert("", "end", values=transaction)

        else:
            messagebox.showerror("Error", "No Transactions")
    
    except Exception as e:
        print("Error loading transactions:", e)

In [392]:
def delete_record():
    selected_item = tree.selection()  

    if not selected_item:
        messagebox.showerror("Error", "Please select a record to delete.")
        return

    item = tree.item(selected_item, "values")  

    if item:
        try:
            conn = sqlite3.connect(db_path) 
            cursor = conn.cursor()

            confirmation = messagebox.askyesno("Delete Record", "Are you sure you want to delete this record?")

            if confirmation:
                if int(item[4]) > 0: 
                    temp_data = (item[0], item[1], item[2], item[3], int(item[4]), int(item[6]))
                    cursor.execute(
                        "DELETE FROM transactions WHERE item_name = ? AND party_name = ? AND item_date = ? AND item_size = ? AND item_in_stock = ? AND balance = ?",
                        temp_data
                    )
                elif int(item[5]) > 0:  # Check for Out Stock
                    temp_data = (item[0], item[1], item[2], item[3], int(item[5]), int(item[6]))
                    cursor.execute(
                        "DELETE FROM transactions WHERE item_name = ? AND party_name = ? AND item_date = ? AND item_size = ? AND item_out_stock = ? AND balance = ?",
                        temp_data
                    )
                else:
                    messagebox.showerror("Error", "Invalid data selection for deletion.")
                    return

                conn.commit() 
                load_transactions() 
                messagebox.showinfo("Success", "Record Deleted Successfully")

        except sqlite3.Error as e:
            messagebox.showerror("Database Error", str(e))

        finally:
            conn.close() 


In [393]:
def search_transaction():
    search_query = search_entry.get().strip()
    if not search_query:
        messagebox.showerror("Error", "Enter item name to search!")
        return

    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT item_name, party_name, item_date, item_size, item_in_stock, item_out_stock, balance FROM transactions WHERE item_name LIKE ?", ('%' + search_query + '%',))
        transactions = cursor.fetchall()

        for item in tree.get_children():
            tree.delete(item)

        if not transactions:
            messagebox.showinfo("Info", "No matching transactions found.")

        for transaction in transactions:
            tree.insert("", "end", values=transaction)

    except sqlite3.Error as e:
        messagebox.showerror("Database Error", str(e))

    finally:
        conn.close()


In [394]:
def open_transactions_page():
    transactions_window = tk.Toplevel(root)
    transactions_window.title("Transaction Reports")

    screen_width = transactions_window.winfo_screenwidth()
    screen_height = transactions_window.winfo_screenheight()
    transactions_window.geometry(f"{screen_width}x{screen_height}+0+0")

    all_transactions_frame = tk.Frame(transactions_window)
    all_transactions_frame.pack(expand=True, fill=tk_constants.BOTH, padx=10, pady=10)

    all_transactions_fields_frame = tk.Frame(all_transactions_frame)
    all_transactions_fields_frame.pack(side=tk.TOP, pady=10)
    
    tk.Label(all_transactions_fields_frame, text="Select Date:").grid(row=0, column=0, padx=5)
    date_entry = DateEntry(all_transactions_fields_frame, width=15, background='darkblue', foreground='white', borderwidth=2)
    date_entry.grid(row=0, column=1, padx=5)

    Purchase_Report_Button = tk.Button(all_transactions_fields_frame, text="Purchase Report", command=lambda: purchase_report( all_transactions_tree, date_entry.get_date().strftime("%d-%m-%Y")))
    Purchase_Report_Button.grid(row=0, column=2, padx=5)

    Sales_Report_Button = tk.Button(all_transactions_fields_frame, text="Sales Report", command=lambda: sales_report(all_transactions_tree, date_entry.get_date().strftime("%d-%m-%Y")))
    Sales_Report_Button.grid(row=0, column=3, padx=5)

    all_transactions_tree = ttk.Treeview(all_transactions_frame, columns=("item_date", "item_name", "party_name", "M", "L", "XL","XXL","3XL","4XL","5XL","FREE","PLUS", "Total"))
    all_transactions_tree.pack(expand=True, fill=tk_constants.BOTH, padx=10, pady=10)

    all_transactions_tree_yscrollbar = ttk.Scrollbar(all_transactions_frame, orient="vertical", command=all_transactions_tree.yview)
    all_transactions_tree_yscrollbar.pack(side=tk_constants.RIGHT, fill=tk_constants.Y)
    all_transactions_tree.configure(yscrollcommand=all_transactions_tree_yscrollbar.set)

    columns = ("item_date", "item_name", "party_name", "M", "L", "XL","XXL","3XL","4XL","5XL","FREE","PLUS", "Total")
    column_widths = (140, 140, 140, 70, 70, 70, 70,70,70,70,70,70,120)

    for col, width in zip(columns, column_widths):
        all_transactions_tree.column(col, width=width)
        all_transactions_tree.heading(col, text=col.capitalize())

    all_transactions_tree.heading("item_name", text="Item Name")
    all_transactions_tree.heading("party_name", text="Party Name")
    all_transactions_tree.heading("item_date", text="Date")
    all_transactions_tree.heading("M", text="M")
    all_transactions_tree.heading("L", text="L")
    all_transactions_tree.heading("XL", text="XL")
    all_transactions_tree.heading("XXL", text="XXL")
    all_transactions_tree.heading("3XL", text="3XL")
    all_transactions_tree.heading("4XL", text="4XL")
    all_transactions_tree.heading("5XL", text="5XL")
    all_transactions_tree.heading("FREE", text="FREE")
    all_transactions_tree.heading("PLUS", text="PLUS")
    all_transactions_tree.heading("Total", text="Total")

In [395]:
root = tk.Tk()
root.title("Inventory Management")

form_frame = tk.Frame(root)
form_frame.grid(row=0, rowspan=4, column=0, padx=10, pady=10, sticky="nsew")

tk.Label(form_frame, text="Date:(mm/dd/yy)").grid(row=0, column=0, sticky="e", pady=5)
cal = DateEntry(form_frame, width=15, background='darkblue', foreground='white', borderwidth=2)
cal.grid(row=0, column=1, pady=5)

tk.Label(form_frame, text="Item Name:").grid(row=1, column=0, sticky="e", pady=5)
item_name_entry_combobox = ttk.Combobox(form_frame)
item_name_entry_combobox.grid(row=1, column=1, sticky="w", pady=5)

tk.Label(form_frame, text="Party Name:").grid(row=2, column=0, sticky="e", pady=5)
party_name_entry_combobox = ttk.Combobox(form_frame)
party_name_entry_combobox.grid(row=2, column=1, sticky="w", pady=5)

tk.Label(form_frame, text="Size:").grid(row=3, column=0, sticky="e", pady=5)
sizes = ['M', 'L', 'XL', 'XXL', '3XL', '4XL', '5XL', 'Free', 'Plus']
size_combobox = ttk.Combobox(form_frame, values=sizes)
size_combobox.grid(row=3, column=1, sticky="w", pady=5)

tk.Label(form_frame, text="In Stock:").grid(row=4, column=0, sticky="e", pady=5)
in_stock_var = tk.StringVar()
in_stock_var.trace_add('write', update_out_stock_state)
in_stock_entry = tk.Entry(form_frame, textvariable=in_stock_var)
in_stock_entry.grid(row=4, column=1, sticky="w", pady=5)
tk.Label(form_frame, text="Out Stock:").grid(row=5, column=0, sticky="e", pady=5)
out_stock_var = tk.StringVar()
out_stock_var.trace_add('write', update_out_stock_state)
out_stock_entry = tk.Entry(form_frame, textvariable=out_stock_var)
out_stock_entry.grid(row=5, column=1, sticky="w", pady=5)

for i in range(6):
    form_frame.rowconfigure(i, weight=1)
form_frame.columnconfigure(0, weight=1)
form_frame.columnconfigure(1, weight=1)

submit_button = tk.Button(form_frame, text="Submit", command=submit_form)
submit_button.grid(row=6, column=0, columnspan=2, pady=10)

Delete_button = tk.Button(form_frame, text="Delete", command=delete_record)
Delete_button.grid(row=7, column=0, columnspan=2, pady=10)

panel_frame = tk.Frame(root)
panel_frame.grid(row=0, rowspan=5, column=1, padx=10, pady=10, sticky="nsew")

filters_frame = tk.Frame(panel_frame)
filters_frame.grid(row=0, column=0, pady=10, sticky="nsew")

tk.Label(filters_frame, text="Filter by Date(mm/dd/yy):").grid(row=1, column=0, padx=5, pady=5)
filter_date_entry = DateEntry(filters_frame, width=15, background='darkblue', foreground='white', borderwidth=2, state = "normal")
filter_date_entry.grid(row=1, column=1, padx=5, pady=5)

tk.Label(filters_frame, text="Filter by Item Name:").grid(row=1, column=2, padx=5, pady=5)
filter_item_name_entry = tk.Entry(filters_frame)
filter_item_name_entry.grid(row=1, column=3, padx=5, pady=5)

filter_button = tk.Button(filters_frame, text="Filter Transactions", command=filter_transaction)
filter_button.grid(row=2, column=0, padx=5, pady=10)

tree = ttk.Treeview(panel_frame, columns=("item_name", "party_name" , "item_date", "item_size", "item_in_stock", "item_out_stock", "balance"))
tree.grid(row=1, rowspan=6, column=0, padx=10, pady=10, sticky="nsew")

tree_yscrollbar = ttk.Scrollbar(panel_frame, orient="vertical", command=tree.yview)
tree_yscrollbar.grid(row=0, rowspan=5, column=1, sticky="ns")
tree.configure(yscrollcommand=tree_yscrollbar.set)


columns = ("item_name", "party_name" , "item_date", "item_size", "item_in_stock", "item_out_stock", "balance")
column_widths = (140, 140, 140, 140, 140, 140, 140)

for col, width in zip(columns, column_widths):
    tree.column(col, width=width)
    tree.heading(col, text=col.capitalize())

tree.heading("item_name", text="Item Name")
tree.heading("party_name", text="Party Name")
tree.heading("item_date", text="Date(dd-mm-yyyy)")
tree.heading("item_size", text="Size")
tree.heading("item_in_stock", text="In Stock")
tree.heading("item_out_stock", text="Out Stock")
tree.heading("balance", text="Balance")

tree.bind("<ButtonRelease-1>", on_tree_select)

totals_frame = tk.Frame(panel_frame)
totals_frame.grid(row=7, column=0, pady=10)

Total_In_Stock = tk.Button(totals_frame, text="Today In Stock", command=update_in_stock_transact)
Total_In_Stock.grid(row=0, column=0, pady=10)

Total_Out_Stock = tk.Button(totals_frame, text="Today Out Stock", command=update_out_stock_transact)
Total_Out_Stock.grid(row=0, column=1, pady=10)

Show_All_Transactions = tk.Button(totals_frame, text="All Transactions", command=load_transactions)
Show_All_Transactions.grid(row=0, column=2, pady=10)

Show_Reports = tk.Button(totals_frame, text="Show Reports", command=open_transactions_page)
Show_Reports.grid(row=0, column=3, pady=10)

for i in range(7):
    panel_frame.rowconfigure(i, weight=1)
panel_frame.columnconfigure(0, weight=1)

load_transactions()

for i in range(6):
    root.rowconfigure(i, weight=1)
    root.columnconfigure(0, weight=1)
    root.columnconfigure(1, weight=1)


if __name__ == "__main__":
    load_transactions()  # Load transactions before GUI starts
    update_dropdowns()   # Ensure dropdowns are updated
    root.mainloop()      # Start the main event loop




